In [2]:
# In terminal, enter: 
# jupyter nbextension enable --py --sys-prefix widgetsnbextension
# pip install gmaps
# jupyter nbextension enable --py --sys-prefix gmaps


import requests
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from project import email
from project import epa_key
from project import gkey
import json
import gmaps

In [5]:
# call annual PM25 2012 dataset (with EPA calculated average values)
# 06 = California
# 36 = New York
# 48 = Texas
# Data year = 2020
list_state = ['48', '36', '06']
df_annual = pd.DataFrame()
for state in list_state:
    try:
        url_2 = 'https://aqs.epa.gov/data/api/annualData/byState?'
        params = {'email': email,
                 'key': epa_key,
                 'param': '88101',
                 'bdate':'20200101',
                 'edate':'20201201',
                 'state': state}

        response = requests.get(url_2, params=params).json()
        data=response['Data']
        int_df = pd.DataFrame(data)
        df_annual = df_annual.append(int_df)
    except:
        pass
df_annual

,state_code,county_code,site_number,parameter_code,poc,latitude,longitude,datum,parameter,sample_duration_code,...,fiftieth_percentile,tenth_percentile,local_site_name,site_address,state,county,city,cbsa_code,cbsa,date_of_last_change
0,48,375,0320,88101,2,35.201592,-101.909275,WGS84,PM2.5 - Local Conditions,X,...,4.7,2.0,Amarillo A&M,6500 Amarillo Blvd West,Texas,Potter,Amarillo,11100,"Amarillo, TX",2021-10-31
1,48,375,0320,88101,2,35.201592,-101.909275,WGS84,PM2.5 - Local Conditions,X,...,4.7,2.0,Amarillo A&M,6500 Amarillo Blvd West,Texas,Potter,Amarillo,11100,"Amarillo, TX",2021-10-31
2,48,375,0320,88101,2,35.201592,-101.909275,WGS84,PM2.5 - Local Conditions,X,...,4.7,2.0,Amarillo A&M,6500 Amarillo Blvd West,Texas,Potter,Amarillo,11100,"Amarillo, TX",2021-10-31
3,48,375,0320,88101,2,35.201592,-101.909275,WGS84,PM2.5 - Local Conditions,X,...,4.7,2.0,Amarillo A&M,6500 Amarillo Blvd West,Texas,Potter,Amarillo,11100,"Amarillo, TX",2021-10-31
4,48,375,0320,88101,2,35.201592,-101.909275,WGS84,PM2.5 - Local Conditions,X,...,4.7,2.0,Amarillo A&M,6500 Amarillo Blvd West,Texas,Potter,Amarillo,11100,"Amarillo, TX",2021-10-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1492,06,001,0013,88101,3,37.864767,-122.302741,NAD83,PM2.5 - Local Conditions,X,...,8.7,4.7,Berkeley Aquatic Park,1 Bolivar Dr,California,Alameda,None,41860,"San Francisco-Oakland-Hayward, CA",2021-10-31
1493,06,001,0013,88101,3,37.864767,-122.302741,NAD83,PM2.5 - Local Conditions,X,...,8.7,4.7,Berkeley Aquatic Park,1 Bolivar Dr,California,Alameda,None,41860,"San Francisco-Oakland-Hayward, CA",2021-10-31
1494,06,001,0013,88101,3,37.864767,-122.302741,NAD83,PM2.5 - Local Conditions,X,...,8.7,4.7,Berkeley Aquatic Park,1 Bolivar Dr,California,Alameda,None,41860,"San Francisco-Oakland-Hayward, CA",2021-10-31
1495,06,001,0013,88101,3,37.864767,-122.302741,NAD83,PM2.5 - Local Conditions,X,...,8.7,4.7,Berkeley Aquatic Park,1 Bolivar Dr,California,Alameda,None,41860,"San Francisco-Oakland-Hayward, CA",2021-10-31


In [6]:
# df_annual.to_csv('/Users/christinenewkirk/Documents/coursework/usc-la-virt-data-pt-09-2021-u-c/air_quality_analysis/CNanalysis_annual')

In [7]:
# remove all rows from the annual dataframe where "pollutant standard" does NOT EQUAL "PM25 Annual 2012"
# this new dataframe will be used for analysis
PM25_annual_2012 = df_annual.loc[df_annual['pollutant_standard'] == 'PM25 Annual 2012']
PM25_annual_2012['pollutant_standard']

2       PM25 Annual 2012
9       PM25 Annual 2012
16      PM25 Annual 2012
23      PM25 Annual 2012
29      PM25 Annual 2012
              ...       
1466    PM25 Annual 2012
1473    PM25 Annual 2012
1480    PM25 Annual 2012
1487    PM25 Annual 2012
1494    PM25 Annual 2012
Name: pollutant_standard, Length: 311, dtype: object

In [8]:
# This dataframe has 311 entries -- how many unique site numbers are there?
unique_sites_annual = PM25_annual_2012.site_number.unique()
unique_sites_annual.sort()
# print(len('unique_sites_annual'))

In [18]:
# seems the most unique identifier is lat + lng, and there are still multiple entries for each of these combos.
# it looks like the same site has multiple entries under "event type." Here is the description of this variable https://aqs.epa.gov/aqsweb/documents/about_aqs_data.html (section 4.2.2)
# select only records where "event type" is "events excluded" or "no event"
array=['Events Excluded', 'No Events']
PM25_annual_2012_final = PM25_annual_2012.loc[PM25_annual_2012['event_type'].isin(array)]

print(PM25_annual_2012_final)

     state_code county_code site_number parameter_code  poc   latitude  \
2            48         375        0320          88101    2  35.201592   
9            48         453        0014          88101    1  30.354944   
16           48         453        0021          88101    3  30.263204   
23           48         453        0021          88101    2  30.263204   
29           48         453        1068          88101    2  30.353860   
...         ...         ...         ...            ...  ...        ...   
1466         06         019        2009          88101    3  36.634225   
1473         06         047        0003          88101    3  37.281853   
1480         06         067        5003          88101    3  38.494475   
1487         06         087        1005          88101    3  37.063150   
1494         06         001        0013          88101    3  37.864767   

       longitude  datum                 parameter sample_duration_code  ...  \
2    -101.909275  WGS84  PM2.5 -

In [10]:
# print csv
PM25_annual_2012_final.to_csv("../air_quality_analysis/PM25_2012_final.csv")

In [11]:
# extract pollutant level, latitude and longitude, for each sensor, as a tuple
location_df = pd.read_csv("../air_quality_analysis/PM25_2012_final.csv")
location_df.head()

sensor_info = list(zip(location_df['arithmetic_mean'],location_df['latitude'],location_df['longitude']))

# print(sensor_info)

In [12]:
# convert tuple to dataframe
for index, tuple in enumerate(sensor_info):
    pollutant_level = tuple[0]
    latitude = tuple[1]
    longitude = tuple[2]

sensor_coords = pd.DataFrame(sensor_info, columns=['Pollutant Level', 'Lat', 'Lng'])

print(sensor_coords)

     Pollutant Level        Lat         Lng
0           5.401760  35.201592 -101.909275
1           5.896053  30.354944  -97.761803
2           9.570787  30.263204  -97.712891
3           8.587500  30.263204  -97.712891
4           9.398847  30.353860  -97.691660
..               ...        ...         ...
240        11.520455  36.634225 -120.382331
241        14.890476  37.281853 -120.433671
242        11.805249  38.494475 -121.211131
243        11.134286  37.063150 -122.083092
244        11.784078  37.864767 -122.302741

[245 rows x 3 columns]


In [13]:
# create a map using pollutant levels and geographic coordinates
gmaps.configure(api_key=gkey)

sensor_coords = sensor_coords.dropna()

locations = sensor_coords[["Lat", "Lng"]].astype(float)
pollutant_level = sensor_coords["Pollutant Level"].astype(float)


In [15]:
# create pollutant level layer
fig = gmaps.figure(layout={
        'width': '800px',
        'height': '600px',
        'padding': '3px',
        'border': '1px solid black'
})

pollutants = gmaps.heatmap_layer(locations, weights=pollutant_level, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(pollutants)

fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='3px', width='800px'))

In [16]:
# add sensor markers
marker_locations = sensor_coords[['Lat', 'Lng']]

fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))

In [17]:
fig = gmaps.figure()

fig.add_layer(pollutants)
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))

In [25]:
state_means = pd.DataFrame(PM25_annual_2012_final, columns=['state_code', 'arithmetic_mean'])
print(state_means)

     state_code  arithmetic_mean
2            48         5.401760
9            48         5.896053
16           48         9.570787
23           48         8.587500
29           48         9.398847
...         ...              ...
1466         06        11.520455
1473         06        14.890476
1480         06        11.805249
1487         06        11.134286
1494         06        11.784078

[245 rows x 2 columns]


In [26]:
state_means.groupby('state_code').mean()

,arithmetic_mean
state_code,
06,11.959265
36,6.357681
48,8.818043


In [ ]:
# From the original 2020 PM 2.5 annual dataset provided by the EPA, records for CA, NY and TX pollutant standard 
# "PM2.5 annual 2012" and event type "Events Excluded" and "No Events" were selected, in order to derive a set of 
# comparble arithmetic mean values of pollutant levels at each sensor.

# Google Maps API was used to create a heatmap of annual average PM2.5 levels in California, Texas and New York in 2020.

# A visual scan of the heatmaps suggests that the average daily PM2.5 concentration in California was worse than in 
# Texas and New York. In fact, the annual arithmetic mean in California was 11.96, in New York, 8.81, and Texas, 6.35.

# The most updated EPA PM2.5 upper limit standard is an annual mean of 12 micrograms per meter cubed. In 2020, as a state, California
# was very close to this upper limit.

